In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report

# Define data directories
data_dir = 'C:/Users/SSC/OneDrive/Desktop/Phase 3/Dataset'  # Replace with the actual path to your data directory
train_dir = os.path.join(data_dir, 'Training')
test_dir = os.path.join(data_dir, 'Testing')
validation_dir = os.path.join(data_dir, 'Validation')


In [3]:
# Define data split ratios
train_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32


In [4]:
# Function to create and compile the CNN model
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [5]:
# Create data generators with data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generators for validation and testing (no data augmentation)
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)


In [6]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)


Found 4200 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [7]:
# Create and train the model
model = create_model()
epochs = 5  # You can adjust the number of epochs
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)


Epoch 1/5
132/132 [==============================] - 98s 731ms/step - loss: 0.6791 - accuracy: 0.5895 - val_loss: 0.6036 - val_accuracy: 0.7117
Epoch 2/5
132/132 [==============================] - 106s 804ms/step - loss: 0.6170 - accuracy: 0.6707 - val_loss: 0.5139 - val_accuracy: 0.7842
Epoch 3/5
132/132 [==============================] - 99s 749ms/step - loss: 0.5962 - accuracy: 0.6900 - val_loss: 0.5051 - val_accuracy: 0.7725
Epoch 4/5
132/132 [==============================] - 101s 767ms/step - loss: 0.5814 - accuracy: 0.7060 - val_loss: 0.4408 - val_accuracy: 0.7925
Epoch 5/5
132/132 [==============================] - 114s 860ms/step - loss: 0.5636 - accuracy: 0.7200 - val_loss: 0.4216 - val_accuracy: 0.8225


In [8]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_accuracy}")


19/19 [==============================] - 7s 379ms/step - loss: 0.4302 - accuracy: 0.8283
Test Accuracy: 0.82833331823349


In [9]:
# Import precision_score, recall_score, f1_score, and accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Make predictions on the test set
predictions = model.predict(test_generator)
predicted_classes = np.round(predictions)

# Generate a classification report
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

print(f"Test Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)


19/19 [==============================] - 3s 146ms/step
Test Accuracy: 0.4716666666666667
Precision: 0.4717607973421927
Recall: 0.47333333333333333
F1-Score: 0.4725457570715474
              precision    recall  f1-score   support

           F       0.47      0.47      0.47       300
           M       0.47      0.47      0.47       300

    accuracy                           0.47       600
   macro avg       0.47      0.47      0.47       600
weighted avg       0.47      0.47      0.47       600



In [10]:
# Updated model architecture
def create_updated_model():
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [11]:
# Create and train the updated model
updated_model = create_updated_model()
epochs = 5  # Increase the number of epochs
history = updated_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)


Epoch 1/5
132/132 [==============================] - 264s 2s/step - loss: 0.6930 - accuracy: 0.5595 - val_loss: 0.5691 - val_accuracy: 0.7133
Epoch 2/5
132/132 [==============================] - 248s 2s/step - loss: 0.6204 - accuracy: 0.6712 - val_loss: 0.6069 - val_accuracy: 0.7242
Epoch 3/5
132/132 [==============================] - 258s 2s/step - loss: 0.6061 - accuracy: 0.6888 - val_loss: 0.5149 - val_accuracy: 0.7342
Epoch 4/5
132/132 [==============================] - 247s 2s/step - loss: 0.5817 - accuracy: 0.7079 - val_loss: 0.4469 - val_accuracy: 0.7983
Epoch 5/5
132/132 [==============================] - 249s 2s/step - loss: 0.5617 - accuracy: 0.7238 - val_loss: 0.4587 - val_accuracy: 0.7958


In [12]:
# Evaluate the updated model
test_loss, test_accuracy = updated_model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_accuracy}")


19/19 [==============================] - 6s 289ms/step - loss: 0.4512 - accuracy: 0.8117
Test Accuracy: 0.8116666674613953


In [13]:
# Make predictions and generate a classification report
predictions = updated_model.predict(test_generator)
predicted_classes = np.round(predictions)

accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

print(f"Test Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)


19/19 [==============================] - 6s 306ms/step
Test Accuracy: 0.48833333333333334
Precision: 0.4885245901639344
Recall: 0.49666666666666665
F1-Score: 0.4925619834710743
              precision    recall  f1-score   support

           F       0.49      0.48      0.48       300
           M       0.49      0.50      0.49       300

    accuracy                           0.49       600
   macro avg       0.49      0.49      0.49       600
weighted avg       0.49      0.49      0.49       600

